In [6]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from pydantic import BaseModel

# Carrega variáveis do .env
load_dotenv()

# Modelo LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Ferramenta simples
@tool
def somar(valores: str) -> str:
    """Soma dois números no formato 'numero1,numero2'."""
    try:
        n1, n2 = map(float, valores.split(","))
        return str(n1 + n2)
    except Exception:
        return "Formato inválido. Use: numero1,numero2"

# Define o schema do estado com pydantic
class AgentState(BaseModel):
    input: str
    resposta: str = None

# Função que processa o input com o LLM
def chamar_llm(state: AgentState) -> AgentState:
    resposta = llm.invoke(state.input)
    return AgentState(input=state.input, resposta=resposta.content)

# Monta o grafo
builder = StateGraph(AgentState)
builder.add_node("llm", chamar_llm)
builder.set_entry_point("llm")
builder.add_edge("llm", END)

# Compila o grafo
graph = builder.compile()

# Testa o agente
entrada = AgentState(input="O que é um agente autônomo de IA?")
resposta = graph.invoke(entrada)
print("Resposta:", resposta["resposta"])

Resposta: Um agente autônomo de IA é um programa de computador que é capaz de tomar decisões e realizar ações de forma autônoma, sem a necessidade de intervenção humana. Esses agentes são projetados para realizar tarefas específicas de forma eficiente e eficaz, utilizando algoritmos de inteligência artificial para aprender e se adaptar ao ambiente em que estão inseridos. Eles podem ser utilizados em uma variedade de aplicações, como assistentes virtuais, sistemas de recomendação, robôs autônomos, entre outros.
